In [1]:
import pandas as pd
from grid_search_utils import get_grid_search_results
import itertools

In [2]:
# give directory name
directory = "test"
get_grid_search_results(directory)

    attributes  values  game_size  vocab_size_factor  n_epochs  batch_size  \
0          3.0     4.0       10.0                3.0      60.0        32.0   
1          3.0     4.0       10.0                3.0      60.0        32.0   
2          3.0     4.0       10.0                3.0      60.0        64.0   
3          3.0     4.0       10.0                3.0      60.0        64.0   
4          3.0     4.0       10.0                3.0      60.0        64.0   
..         ...     ...        ...                ...       ...         ...   
91         5.0    16.0       10.0                3.0      60.0       128.0   
92         5.0    16.0       10.0                3.0      60.0       128.0   
93         5.0    16.0       10.0                3.0      60.0       128.0   
94         5.0    16.0       10.0                3.0      60.0       128.0   
95         5.0    16.0       10.0                3.0      60.0       128.0   

    learning_rate  hidden_size  temp_update  temperature  train

In [3]:
data = pd.read_csv("results_" + directory + ".csv")

## New grid search results

In [4]:
# parameters that were tested
for col in data.columns[:-4]:
    print(col, data[col].unique())    

attributes [3 5]
values [ 4 16]
game_size [10]
vocab_size_factor [3.]
n_epochs [60.]
batch_size [ 32  64 128]
learning_rate [0.0005 0.001 ]
hidden_size [128 256]
temp_update [0.97 0.99]
temperature [2.]


In [5]:
data = data[((data['attributes'] == 3) & (data['values'] == 4))
            | ((data['attributes'] == 3) & (data['values'] == 16))
            | ((data['attributes'] == 5) & (data['values'] == 4))]

In [6]:
attributes = [3, 5]
values = [4, 16]
datasets = list(itertools.product(attributes, values))[:3]
index = data.index
for (attributes, values) in datasets:
    print("dataset:", attributes, values)
    per_dataset = data[(data["attributes"] == attributes) & (data["values"] == values)]
    max_test_acc = per_dataset['test_accuracy'].idxmax()
    print(per_dataset.loc[index[max_test_acc]])

dataset: 3 4
attributes             3.000000
values                 4.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          256.000000
temp_update            0.990000
temperature            2.000000
train_loss             0.109561
train_accuracy         0.964298
test_loss              0.344927
test_accuracy          0.915234
Name: 7, dtype: float64
dataset: 3 16
attributes             3.000000
values                16.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          256.000000
temp_update            0.970000
temperature            2.000000
train_loss             0.162047
train_accuracy         0.945292
test_loss              0.319436
test_accuracy          0.906445
Name: 30, dtype: float64
dataset: 5 4
attributes     

### Batch size
Highest mean test accuracies with batch size 64, highest maximum test accuracies with batch size 32.

In [7]:
bs_sumstat = data.groupby(['attributes', 'values', 'batch_size'])['train_accuracy', 'test_accuracy'].describe()
bs_train = bs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
bs_test = bs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
bs_merged = bs_train.merge(bs_test, left_index=True, right_index=True)
bs_merged

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

### Learning rate
Higher maximum accuracies with learning rate 0.001, but higher mean accuracies with learning rate 0.0005.

In [8]:
lr_sumstat = data.groupby(['attributes', 'values', 'learning_rate'])['train_accuracy', 'test_accuracy'].describe()
lr_train = lr_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
lr_test = lr_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
lr_merged = lr_train.merge(lr_test, left_index=True, right_index=True)
lr_merged

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

### Hidden size
Highest mean accuracies with hidden size 128.

In [9]:
hs_sumstat = data.groupby(['attributes', 'values', 'hidden_size'])['train_accuracy', 'test_accuracy'].describe()
hs_train = hs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
hs_test = hs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
hs_merged = hs_train.merge(hs_test, left_index=True, right_index=True)
hs_merged

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

### Temperature
Highest mean accuracies with temperature 2.

In [10]:
t_sumstat = data.groupby(['attributes', 'values', 'temperature'])['train_accuracy', 'test_accuracy'].describe()
t_train = t_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
t_test = t_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
t_merged = t_train.merge(t_test, left_index=True, right_index=True)
t_merged

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

### Temperature update
Highest mean accuracies for temperature update 0.99.

In [11]:
tu_sumstat = data.groupby(['attributes', 'values', 'temp_update'])['train_accuracy', 'test_accuracy'].describe()
tu_train = tu_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
tu_test = tu_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
tu_merged = tu_train.merge(tu_test, left_index=True, right_index=True)
tu_merged

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.